In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
from stockapp import models as stkmd
from dataapp import models as dtamd
from dataapp import tasks as dtatks
from stockapp import tasks as stktks
import utility as uty
import itertools as itt
import multiprocessing as mp
from django.db import connection,connections
from django.db import reset_queries
import time 
import pandas as pd
import numpy as np

In [7]:
semaphore = mp.Semaphore(1)

def func(x,*args,**kwargs):
    time.sleep(1)
    with kwargs['semaphore']:
        print x
        time.sleep(2)
P=[]
for i in range(10):
    P.append(mp.Process(target=func,args=(i,),kwargs={'semaphore':semaphore}))

for p in P:
    p.start()
    
for p in P:
    p.join()

0
1
2
3
4
6
7
5
8
9


print dtamd.Stockprice.objects.all().delete()
stkmd.Stockmeta.objects.all().update(LastPriceUpdate=None,Lastdate=None,Startdate=None)


In [ ]:
tt=(1,2)
tt=tt+(3,)
tt

In [ ]:
print stkmd.Stockmeta.objects.filter(Symbol='AAPL').values()
stk=stkmd.Stockmeta.objects.get(Symbol='AAPL')
print stk
print dtamd.Stockprice.objects.filter(Symbol='AAPL').count()
dtatks.UpdatePriceData([stk.id],inputtype='id')

In [ ]:
stk=stkmd.Stockmeta.objects.get(Symbol='AAPL')
stocksiter=iter([[stk.id,stk.id],[stk.id]])
# for ss in stocksiter:
#     print tuple(ss)
# print 
computeargs_iter=itt.imap(lambda x: itt.izip([x],['id'] ) , stocksiter)
for ss in computeargs_iter:
    print list( ss )

In [ ]:
for ss in zip([(1,2),(3,4)],itt.repeat('id')):
    print ss

In [ ]:
# run in parallel
stk=stkmd.Stockmeta.objects.get(Symbol='AAPL')
stocksiter=iter([[stk.id,stk.id],[stk.id]])
computeargs_iter=itt.izip(stocksiter,itt.repeat('id') ) 
      
PllCmpt=uty.ParallelCompute( computeargs_iter, dtatks.UpdatePriceData )
PllCmpt.parallelrun()

In [8]:
dtatks.RunDataDownload()

error downloading  ARSD  for input dates  2002-01-01 2017-05-01
error downloading  CMGE  for input dates  2002-01-01 2017-05-01
error downloading  IG  for input dates  2002-01-01 2017-05-01
error downloading  ICGE  for input dates  2002-01-01 2017-05-01
error downloading  LEAF  for input dates  2002-01-01 2017-05-01
error downloading  PCP  for input dates  2002-01-01 2017-05-01
error downloading  SZYM  for input dates  2002-01-01 2017-05-01
error downloading  BEE  for input dates  2002-01-01 2017-05-01
error downloading  TLM  for input dates  2002-01-01 2017-05-01
error downloading  TEG  for input dates  2002-01-01 2017-05-01
error downloading  CEP  for input dates  2002-01-01 2017-05-01
error downloading  HNSN  for input dates  2002-01-01 2017-05-01
error downloading  SVLC  for input dates  2002-01-01 2017-05-01
error downloading  GSJK  for input dates  2002-01-01 2017-05-01
error downloading  VCLK  for input dates  2002-01-01 2017-05-01
error downloading  GVT  for input dates  2002-0

Process Process-126:
Process Process-150:
Process Process-151:
Process Process-154:
Process Process-157:
Process Process-133:
Process Process-155:
Process Process-134:
Process Process-153:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-141:
Traceback (most recent call last):
Process Process-127:
Traceback (most recent call last):
Process Process-139:
Process Process-131:
Process Process-128:
Process Process-146:
Process Process-156:
Process Process-88:
KeyboardInterrupt
Process Process-158:


Process Process-160:
Process Process-130:
Process Process-129:
Process Process-135:
Process Process-143:
Process Process-159:
Process Process-116:
Process Process-147:
Process Process-124:
Process Process-145:
Process Process-97:
Process Process-132:
Process Process-121:
Process Process-86:
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-138:
Process Process-125:
Process Process-142:
Process Process-148:
Process Process-122:
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process Process-120:
Process Process-117:
Process Process-118:
Process Process-115:
Process Process-152:
Process Process-149:
Process Process-144:
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call las

    self.run()
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
  Fi

KeyboardInterrupt: 

...

    return query.get_compiler(using=using).execute_sql(return_id)


  File "/usr/local/lib/python2.7/dist-packages/django/db/models/sql/compiler.py", line 1098, in execute_sql
    kwargs['lock'].acquire()
    data = SingleBlockManager(data, index, fastpath=True)
    for sql, params in self.as_sql():
KeyboardInterrupt
  File "/usr/local/lib/python2.7/dist-packages/pandas/core/internals.py", line 4072, in __init__
    self.blocks = [block]
KeyboardInterrupt


  File "/usr/local/lib/python2.7/dist-packages/django/db/models/sql/compiler.py", line 1051, in as_sql


    for obj in self.query.objs
  File "/usr/local/lib/python2.7/dist-packages/django/db/models/sql/compiler.py", line 990, in prepare_value
    value = field.get_db_prep_save(value, connection=self.connection)


  File "/usr/local/lib/python2.7/dist-packages/django/db/models/fields/__init__.py", line 1600, in get_db_prep_save
    return connection.ops.adapt_decimalfield_value(self.to_python(value), self.max_digits, self.decimal_places)
  File "/usr/local/lib/python2.7/dist-packages/django/db/backends/base/operations.py", line 503, in adapt_decimalfield_value
    return utils.format_number(value, max_digits, decimal_places)
  File "/usr/local/lib/python2.7/dist-packages/django/db/backends/utils.py", line 215, in format_number
    return "{:f}".format(value)
  File "/usr/lib/python2.7/decimal.py", line 3643, in __format__
    def __format__(self, specifier, context=None, _localeconv=None):
KeyboardInterrupt


In [ ]:
stk=stkmd.Stockmeta.objects.get(Symbol='AAPL')
print stk.LastPriceUpdate
print stk.Lastdate
print stk.Startdate

In [ ]:
from dataapp import models as dtamd
stkmd.Stockmeta.objects.get(Symbol='AAPL').values_list()
dtamd.Stockprice.objects.filter(Symbol='AAPL')
dtamd.Stockprice.objects.all().delete()

In [ ]:
dfstockmeta=pd.read_csv('databases/olddata/stockmeta.csv')
dfwatchlist=pd.read_csv('databases/olddata/watchlist.csv')
dfwatchlist_stocks=pd.read_csv('databases/olddata/watchlist_stocks.csv')

In [ ]:
store=pd.HDFStore('databases/olddata/StockData.h5')
store['AAPL']

In [ ]:
for stk in dfstockmeta['Symbol']:
    print stk
    try:
        df=store[stk]
    except:
        continue
    objs=[]
    for ind in df.index:
        objs.append( dtamd.Stockprice(Close=df.loc[ind,'Close'], Open=df.loc[ind,'Open'] ,
                                     High=df.loc[ind,'High'],Low=df.loc[ind,'Low'],
                                     Volume=df.loc[ind,'Volume'],Date=ind,Symbol=stk)  )
    
    dtamd.Stockprice.objects.bulk_create(objs)
  

In [ ]:
ss=stkmd.Stockmeta.objects.all()
S=ss.values('Symbol','Company','Sector','Industry','Status','Labels')
S[0]

In [ ]:
from django.db.models import Count,Max,Min
sp=ss.values('Date').annotate(MaxClose=Max('Close'),MinClose=Min('Close'))
print sp
print
print connections['stockpricedata'].queries
reset_queries()

In [9]:
df=pd.read_sql("""SELECT "dataapp_stockprice"."Date", 
               CAST(MIN("dataapp_stockprice"."Close") AS NUMERIC) AS "MinClose", 
               CAST(MAX("dataapp_stockprice"."Close") AS NUMERIC) AS "MaxClose", 
               CAST(AVG("dataapp_stockprice"."Close") AS NUMERIC) AS "AvgClose" 
               FROM "dataapp_stockprice" 
               WHERE "dataapp_stockprice"."Date" BETWEEN \'2015-01-01\' AND \'2016-01-01\' AND "dataapp_stockprice"."Close" >2 AND "dataapp_stockprice"."Close"<1000
               AND "dataapp_Stockprice"."id" IN (SELECT stockid FROM  )
               GROUP BY "dataapp_stockprice"."Date" 
               ORDER BY "dataapp_stockprice"."Date" ASC LIMIT 21"""
    ,connections['stockpricedata'])

/usr/local/lib/python2.7/dist-packages/django/db/models/base.py:324: RuntimeWarning: Model 'stockapp.stockmeta' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)
/usr/local/lib/python2.7/dist-packages/django/db/models/base.py:324: RuntimeWarning: Model 'stockapp.indexfunction_user' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)
/usr/local/lib/python2.7/dist-packages/django/db/models/base.py:324: RuntimeWarning: Model 'stockapp.indexfunction' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)
/usr/local/lib/python2.7/dist-packages/django/db

DatabaseError: Execution failed on sql 'SELECT "dataapp_stockprice"."Date", 
               CAST(MIN("dataapp_stockprice"."Close") AS NUMERIC) AS "MinClose", 
               CAST(MAX("dataapp_stockprice"."Close") AS NUMERIC) AS "MaxClose", 
               CAST(AVG("dataapp_stockprice"."Close") AS NUMERIC) AS "AvgClose" 
               FROM "dataapp_stockprice" 
               WHERE "dataapp_stockprice"."Date" BETWEEN '2015-01-01' AND '2016-01-01' AND "dataapp_stockprice"."Close" >2 AND "dataapp_stockprice"."Close"<1000
               AND "dataapp_Stockprice"."id" IN (SELECT stockid FROM  )
               GROUP BY "dataapp_stockprice"."Date" 
               ORDER BY "dataapp_stockprice"."Date" ASC LIMIT 21': near ")": syntax error

In [17]:
df=pd.read_sql("""SELECT * from dataapp_stockprice WHERE "dataapp_stockprice"."Date" BETWEEN \'2015-01-01\' AND \'2016-01-01\' AND "dataapp_stockprice"."Close" >2 AND "dataapp_stockprice"."Close"<1000"""
    ,connections['stockpricedata'])

In [19]:
# print df['Symbol'].unique()
df[df['Symbol']=='BSND']

,id,Close,Open,High,Low,Volume,Date,Symbol,Symbol_id
286,1176771,415.0,415.0,415.0,415.0,0.0,2015-01-02,BSND,5005
831,1176772,415.0,415.0,415.0,415.0,0.0,2015-01-05,BSND,5005
1376,1176773,415.0,415.0,415.0,415.0,0.0,2015-01-06,BSND,5005
1922,1176774,415.0,415.0,415.0,415.0,0.0,2015-01-07,BSND,5005
2470,1176775,415.0,415.0,415.0,415.0,0.0,2015-01-08,BSND,5005
3017,1176776,415.0,415.0,415.0,415.0,0.0,2015-01-09,BSND,5005
3564,1176777,415.0,415.0,415.0,415.0,0.0,2015-01-12,BSND,5005
4110,1176778,415.0,415.0,415.0,415.0,0.0,2015-01-13,BSND,5005
4657,1176779,415.0,415.0,415.0,415.0,0.0,2015-01-14,BSND,5005
5204,1176780,415.0,415.0,415.0,415.0,0.0,2015-01-15,BSND,5005


In [ ]:
import pymongo

from pymongo import MongoClient
client = MongoClient()

client = MongoClient('localhost', 27017)
client = MongoClient('mongodb://localhost:27017/')

In [ ]:
db = client['test-database']

collection = db['test-collection']

import datetime
post = {"author": "Mike",
         "text": "My first blog post!",
         "tags": ["mongodb", "python", "pymongo"],
         "date": datetime.datetime.utcnow(),
         "somearray": {"X":[1,2,3],"Y":[4,5,6]} }

posts = db.posts
pp=db.posts.insert_one(post)
pp

In [ ]:
db.collection_names(include_system_collections=True)

In [ ]:
result = db.profiles.create_index([('user_id', pymongo.ASCENDING)],unique=True)
sorted(list(db.profiles.index_information()))


In [ ]:
import numpy as np
db.testing.update_many(
   {  },
   {
     "$set": { "Some": np.nan},
     "$currentDate": { "lastModified": True }
   }
)

In [ ]:
qty="qty"
size="size"
h="h"
w="w"
uom="uom"
status="status"
db.testing.insert_one( 
   { "item": "canvas", qty: 100, size: { h: 28, w: 35.5, uom: "cm" }, status: "A" },
);

db.testing.insert_many( [
   { "item": "canvas", qty: 100, size: { h: 28, w: 35.5, uom: "cm" }, status: "A" },
   { "item": "journal", qty: 25, size: { h: 14, w: 21, uom: "cm" }, status: "A" },
   { "item": "mat", qty: 85, size: { h: 27.9, w: 35.5, uom: "cm" }, status: "A" },
   { "item": "mousepad", qty: 25, size: { h: 19, w: 22.85, uom: "cm" }, status: "P" },
   { "item": "notebook", qty: 50, size: { h: 8.5, w: 11, uom: "in" }, status: "P" },
]);

In [ ]:
for pp in db.testing.find():
    print pp

In [ ]:
pd.read_sql('Select * From dataapp_stockprice',connections['stockpricedata'])

In [ ]:
uty.get_celery_worker_status()

In [ ]:
dtamd.Stockprice.objects.filter(Symbol='AAPL').delete()

In [ ]:
print stkmd.Stockmeta.objects.filter(Symbol='AAPL').values()
stk=stkmd.Stockmeta.objects.get(Symbol='AAPL')
print stk
print dtamd.Stockprice.objects.filter(Symbol='AAPL').count()
dtatks.UpdatePriceData([stk.id],inputtype='id')

In [2]:
import multiprocessing as mp
class Someclass(object):
    def __init__(self,x):
        self.x=x
    def __call__(self,a):
        print a+self.x

p=mp.Process(target=Someclass(3),args=(4,))
p.start()
p.join()

7


In [9]:
from celery import shared_task
class Someclass(object):
    def __init__(self,x):
        self.x=x
    @shared_task
    def __call__(self,a):
        print a+self.x
A=Someclass(3)

@shared_task
def add(x):
    print x+3
r=add.delay(4)
r.collect()

<generator object collect at 0x7f6261934690>